In [9]:
#pip install pytesseract
!pip install pillow

In [17]:
import cv2

In [ ]:
import pytesseract
import cv2
import os
from PIL import Image

pytesseract.pytesseract.tesseract_cmd = R'C:/Program Files/Tesseract-OCR/Tesseract'


#이미지를 불러와 gray 스케일로 변환해 준다.

image = cv2.imread('img11.jpg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

#pytesseract에서는 numpy array를 읽지 못하고 file을 읽기 때문에 os로 파일을 불러들여야 한다.
filename = "{}.jpg".format(os.getpid())
cv2.imwrite(filename, gray)

#pytesseract의 image to string을 써준다. 
#숫자니까 lang = 'None'으로

text = pytesseract.image_to_string(Image.open(filename), lang='kor+eng')
os.remove(filename)

#결과를 보자.

print(text)
cv2.imshow('img',image)
cv2.waitKey(0)

In [ ]:
def selectWords(img):
    # org = cv2.imread('capture4.png', cv2.IMREAD_COLOR)
    org = img
    # org = cv2.resize(org, dsize=(0,0), fx=0.5, fy=0.5)
    gray = cv2.cvtColor(org, cv2.COLOR_BGR2GRAY)  # ================  1 gray scale로 변환

    kernel = np.ones((52, 2), np.uint8)
    kernel2 = np.ones((6, 15), np.uint8)
    roi_list = []

    morph = cv2.morphologyEx(gray, cv2.MORPH_GRADIENT, kernel)  # 2 ================ 경계선 찾기

    thr = cv2.adaptiveThreshold(morph, 255, cv2.ADAPTIVE_THRESH_MEAN_C,  cv2.THRESH_BINARY_INV, 3, 30)  # 3 ================ 임계처리

    morph2 = cv2.morphologyEx(thr, cv2.MORPH_CLOSE, kernel2)  # 4 ================ 뭉게기

    contours, _ = cv2.findContours(morph2, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)  # 5 ================ 특징점 찾기

    org2 = cv2.copyMakeBorder(org, 0, 0, 0, 0, cv2.BORDER_REPLICATE)
    for cnt in contours:
        try:
            x, y, w, h = cv2.boundingRect(cnt)
            if w > 5 and 30 < h < 100:
                # print(w, h)
                roi = org2[y:y + h, x:x + w]
                # cv2.imshow('roi', roi)
                roi_list.append(roi)
                cv2.rectangle(org, (x, y), (x+w, y+h), (255, 0, 0), 2)

        except Exception as e:
           pass

    cnt = 0              # print all pieces
    '''for r in roi_list:
        cnt += 1
        cv2.imshow(str(cnt), r)'''

    cv2.imshow('org', org)
    cv2.imshow('roi_list', roi_list)
    cv2.imshow('gray', gray)
    cv2.imshow('morph', morph)
    cv2.imshow('morph2', morph2)
    cv2.imshow('thr', thr)

    return org, roi_list, gray, morph, morph2, thr